In [2]:
from TexSoup import TexSoup
import shutil
import re
import os
import difflib
import pprint

#os.path.exists(os.path.join(os.getcwd(), 'src/pages/sta'))

In [3]:
coursetags = {
    'Statics': 'sta',
    'Dynamics': 'dyn',
    'Solid Mechanics': 'sol'
}

course = 'Statics'

tex_files = [file[:-4] for file in os.listdir(f'./public/{course}') if file[-4:] == '.tex']

tex_files

['BlankTopic',
 'Cartesian Coordinates',
 'Centers of Mass',
 'ForceSystems',
 'Free Body Diagrams',
 'Friction',
 'Hydrostatic Fluid Pressure',
 'Internal Forces',
 'Introduction',
 'main',
 'Moment of Inertia',
 'Moments',
 'Reaction Forces',
 'Trusses',
 'Vectors Scalars',
 'Virtual Work']

In [4]:
replacements = {
    'section': 'Section',
    'subsection': 'SubSection',
    'subsubsection': 'SubSubSection',
    'blue': 'BlueText',
    'red': 'RedText',
    'textit': 'em',
    'textbf': 'strong'
}

In [26]:
from tex2py import tex2py

with open('./public/Statics/Trusses.tex', 'r') as file:
    data = tex2py(file.read())

    print(data.findHierarchy())
    
    sections = list(data.sections)

    #print(sections)

    subsections = {section: section.branches for section in sections}

    #print(subsections)

('section', 'subsection')


In [7]:
for filename in tex_files:
    required_modules = ['Image']

    tex_path = f"public/{course}/{filename}.tex"

    astro_path = f"src/pages/{coursetags[course]}/{filename.lower().replace(' ', '_')}.astro"


    with open(tex_path, 'r+') as file:
        raw = file.read()
        data = TexSoup(raw)
        for tex, html in replacements.items():
            matches = data.find_all(tex)
            #print(matches, tex)
            for match in matches:
                if len(match.contents) != 0:
                    raw = raw.replace(r"\%s{%s}" % (tex, match.contents[0]), f"<{html}>{match.contents[0]}</{html}>")

                    if html != 'em' and html != 'strong' and (html not in required_modules):
                        required_modules.append(html)
        
        # ADDITIONAL CLEANUP

        
        print([e for e in data.subsection.descendants])
        

        raw = raw.replace(r'\noindent', '')

        for l in data.find_all('enumerate'):
            s = str(l)
            final_str = ''
            for c in l.children:
                final_str += f"\t<li>{c.text[0]}</li>\n"
            
            final_str = '<ol>\n'+final_str+'</ol>\n'

            raw = raw.replace(s, final_str)

        comments = re.findall('\n%(.*)\n', raw)
        
        for comment in comments:
            raw = raw.replace(f'%{comment}', f'<!-- {comment} -->')


        inline_equations = re.findall('\$([^$]*)\$', raw)
        
        if len(inline_equations): required_modules.append('InlineEquation')

        for eq in inline_equations:
            db_backslash = eq.replace('\\','\\\\')
            raw = raw.replace(f'${eq}$', f"<InlineEquation equation='{db_backslash}' />")

        display_equations = re.findall('\[(.*)\]', raw)

        for eq in display_equations:
            db_backslash = eq.replace('\\','\\\\')
            if r'\[%s]'% (eq) in raw and ('DisplayEquation' not in required_modules): required_modules.append('DisplayEquation')
            raw = raw.replace(r'\[%s]'% (eq), f"<DisplayEquation equation='{db_backslash}' />")

        imports = 'import Layout from "../../layouts/Layout.astro" \n'

        for comp in required_modules:
            imports += f'import {comp} from "../../components/{comp}.astro" \n' 

        figs = data.find_all('figure*')

        for fig in figs:
            caption = ' '.join([str(e) for e in fig.caption.contents[1:]])
            
            actual_text = caption[6:-1] if caption[0:5] == r'\blue' else caption
            
            inline_equations = re.findall('\$([^$]*)\$', actual_text)

            for eq in inline_equations:
                db_backslash = eq.replace('\\','\\\\')
                actual_text = actual_text.replace(f'${eq}$', f"<InlineEquation equation='{db_backslash}' />")

            if caption[0:5] == r'\blue':
                actual_text = f'<BlueText>{actual_text}</BlueText>'

            src = fig.includegraphics.text[-1]
            width = fig.includegraphics.text[0].split(',')[1].split('=')[1].replace('in', '')

            print(actual_text)

            raw = raw.replace(r'\end{figure*}', f"<Image src='/{course}/{src}' width='{width}'>{actual_text}</Image>", 1)

        #print(imports)

        raw = '---\n' + imports + '---\n' + f'<Layout title="{filename}">' + '\n' + raw + '\n' +'</Layout>'
        

        astro_file = open(astro_path, mode='w+')

        astro_file.write(raw)

        astro_file.close()



['Component 1']
['Vector Bases']
<BlueText>Taken from TAM 210 Lecture 3 - Slide 3</BlueText>
['Basis Shapes']
 Nonuniform load. Need to remake this in the same style as the other pictures.
['Equivalent systems']
 A concentrated force applied to a bar at point A.
 Example distributed load (top) and its equivalent force system (bottom).
 Example triangular distributed load (top) and its equivalent force system (bottom).
 Nonuniform load. Need to remake this in the same style as the other pictures.
['Pulley Idealizations']
<BlueText>Taken from TAM 210 Lecture 5 - Slide 6</BlueText>
<BlueText>Taken from TAM 210 Lecture 5 - Slide 7. Make this picture and put it next to a stress-strain curve with the straight linear elastic line and the slope as k.</BlueText>
<BlueText>Taken from TAM 210 Lecture 5 - Slide 8. This could become an interactive image eventually where the angles of the surfaces are changed by the user and then the reaction forces are also changing.</BlueText>
[\red{Static Frictio

AttributeError: 'NoneType' object has no attribute 'descendants'

In [34]:
re.findall('\$([^$]*)\$', r'ration $\vec{a}$ of a point mass with mass $m$ to the total applied force $\vec{F}$ on the mass ')

['\\vec{a}', 'm', '\\vec{F}']